In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/Users/awxlong/Desktop/my-studies/hpc_exps/')


In [ ]:
from HistoMIL.MODEL.Image.MIL.CAMIL.model import CAMIL
from HistoMIL.MODEL.Image.MIL.CAMIL.paras import CAMILParas
from HistoMIL.MODEL.Image.MIL.utils import Attention

In [ ]:
import h5py
from sklearn.metrics import pairwise_distances
import numpy as np
from scipy.spatial import distance
import torch
import os
from sklearn import preprocessing
import pickle
import pandas as pd
from HistoMIL import logger
import logging
logger.setLevel(logging.INFO)
import pdb
import numpy as np
from sklearn.metrics import pairwise_distances
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
# class CustomAttentioneEff(nn.Module):
#     def __init__(
#             self,
#             input_dim,
#             weight_params_dim,
#             kernel_initializer="xavier_uniform",
#             kernel_regularizer=None,
#     ):
#         super().__init__()

#         self.weight_params_dim = weight_params_dim

#         # Initialize weights
#         self.wq_weight_params = nn.Parameter(torch.Tensor(input_dim, 
#                                                           weight_params_dim))
#         self.wk_weight_params = nn.Parameter(torch.Tensor(input_dim, 
#                                                           weight_params_dim))

#         # Initialize weights using the specified initializer
#         if kernel_initializer == "xavier_uniform":
#             nn.init.xavier_uniform_(self.wq_weight_params)
#             nn.init.xavier_uniform_(self.wk_weight_params)
#         else:
#             # Add other initializers as needed
#             raise ValueError(f"Unsupported initializer: {kernel_initializer}")

#         # Regularization is typically applied during the loss computation in PyTorch
#         self.kernel_regularizer = kernel_regularizer

#     def forward(self, inputs):
#         return self.compute_attention_scores(inputs)

#     def compute_attention_scores(self, instance):
#         ### can futher optimize this using F.scaled_dot_product 
#         # q = torch.matmul(instance, self.wq_weight_params)
#         # k = torch.matmul(instance, self.wk_weight_params)

#         # # dk = torch.tensor(k.size(-1), dtype=torch.float32)
#         # dk = torch.tensor(k.shape[-1], dtype=torch.int32)
#         # # pdb.set_trace()
#         # matmul_qk = torch.matmul(q, k.transpose(-2, -1))  # (..., seq_len_q, seq_len_k)
#         # # matmul_qk = torch.tensordot(q, k.transpose(-2, -1), dims=1) # could also be this
        
#         # scaled_attention_logits = matmul_qk / torch.sqrt(dk)

        
#         chunk_size = 1024  # Adjust this value based on your GPU memory
#         q_chunks = []
#         k_chunks = []
        
#         for i in range(0, instance.size(0), chunk_size):
#             chunk = instance[i:i+chunk_size]
#             q_chunks.append(torch.matmul(chunk, self.wq_weight_params))
#             k_chunks.append(torch.matmul(chunk, self.wk_weight_params))
        
#         q = torch.cat(q_chunks, dim=0)
#         k = torch.cat(k_chunks, dim=0)

#         # Use torch.sqrt() directly on a scalar
#         dk = torch.sqrt(torch.tensor(k.size(-1), dtype=torch.float32))

#         # Use torch.bmm for batch matrix multiplication
#         # Reshape q and k for bmm
#         q_reshaped = q.view(-1, q.size(-2), q.size(-1))
#         k_reshaped = k.view(-1, k.size(-2), k.size(-1))
        
#         # Compute attention scores in chunks
#         attention_chunks = []
#         for i in range(0, q_reshaped.size(0), chunk_size):
#             q_chunk = q_reshaped[i:i+chunk_size]
#             k_chunk = k_reshaped[i:i+chunk_size]
            
#             matmul_qk = torch.bmm(q_chunk, k_chunk.transpose(1, 2))
#             scaled_chunk = matmul_qk / dk
#             attention_chunks.append(scaled_chunk)
        
#         scaled_attention_logits = torch.cat(attention_chunks, dim=0)
        
#         # Reshape back to original dimensions
#         scaled_attention_logits = scaled_attention_logits.view(q.shape[:-1] + (k.size(-2),))
        
#         return scaled_attention_logits
    
# class CustomAttention(nn.Module):
#     def __init__(
#         self,
#         input_dim,
#         weight_params_dim=256,
#         kernel_initializer="glorot_uniform",
#         kernel_regularizer=None,
#         chunk_size=1024
#     ):
#         super().__init__()

#         self.weight_params_dim = weight_params_dim

#         # Initialize weights
#         self.wq_weight_params = nn.Parameter(torch.Tensor(input_dim, 
#                                                           weight_params_dim))
#         self.wk_weight_params = nn.Parameter(torch.Tensor(input_dim, 
#                                                           weight_params_dim))
#         # Set initializer
#         self.kernel_initializer = kernel_initializer

#         # Set regularizer (L2 regularization in PyTorch)
#         self.kernel_regularizer = kernel_regularizer

#         # self.chunk_size = chunk_size
#         self.reset_parameters()
#     def reset_parameters(self):
#         if self.kernel_initializer == "glorot_uniform":
#             nn.init.xavier_uniform_(self.wq_weight_params)
#             nn.init.xavier_uniform_(self.wk_weight_params)
#         else:
#             nn.init.kaiming_uniform_(self.wq_weight_params, a=torch.sqrt(5))
#             nn.init.kaiming_uniform_(self.wk_weight_params, a=torch.sqrt(5))

#     def forward(self, inputs):
        
        
#         attention_weights = self.compute_attention_scores(inputs)
#         return attention_weights

#     def compute_attention_scores(self, instance):
#         q = torch.matmul(instance, self.wq_weight_params)
#         k = torch.matmul(instance, self.wk_weight_params)

#         # dk = torch.tensor(k.size(-1), dtype=torch.float32)
#         dk = torch.tensor(k.shape[-1], dtype=torch.int32)
#         # pdb.set_trace()
#         matmul_qk = torch.matmul(q, k.transpose(-2, -1))  # (..., seq_len_q, seq_len_k)
#         # matmul_qk = torch.tensordot(q, k.transpose(-2, -1), dims=1) # could also be this
        
#         scaled_attention_logits = matmul_qk / torch.sqrt(dk)
#         return scaled_attention_logits
    
#     def compute_attention_scores_eff(self, instance):
        
#         chunk_size = 1024  # Adjust this value based on your GPU memory
#         q_chunks = []
#         k_chunks = []
        
#         for i in range(0, instance.size(0), chunk_size):
#             chunk = instance[i:i+chunk_size]
#             q_chunks.append(torch.matmul(chunk, self.wq_weight_params))
#             k_chunks.append(torch.matmul(chunk, self.wk_weight_params))
        
#         q = torch.cat(q_chunks, dim=0)
#         k = torch.cat(k_chunks, dim=0)

#         # Use torch.sqrt() directly on a scalar
#         dk = torch.sqrt(torch.tensor(k.size(-1), dtype=torch.float32))

#         # Use torch.bmm for batch matrix multiplication
#         # Reshape q and k for bmm
#         q_reshaped = q.view(-1, q.size(-2), q.size(-1))
#         k_reshaped = k.view(-1, k.size(-2), k.size(-1))
        
#         # Compute attention scores in chunks
#         attention_chunks = []
#         for i in range(0, q_reshaped.size(0), chunk_size):
#             q_chunk = q_reshaped[i:i+chunk_size]
#             k_chunk = k_reshaped[i:i+chunk_size]
            
#             matmul_qk = torch.bmm(q_chunk, k_chunk.transpose(1, 2))
#             scaled_chunk = matmul_qk / dk
#             attention_chunks.append(scaled_chunk)
        
#         scaled_attention_logits = torch.cat(attention_chunks, dim=0)
        
#         # Reshape back to original dimensions
#         scaled_attention_logits = scaled_attention_logits.view(q.shape[:-1] + (k.size(-2),))
        
#         return scaled_attention_logits
    
#     def extra_repr(self):
#         return f'weight_params_dim={self.weight_params_dim}'

In [ ]:
rand_tensor = torch.rand((1, 420, 1024))


In [ ]:
# # mdl = CustomAttention(input_dim=1024, weight_params_dim=256)
# mdl1 = CustomAttentioneEff(input_dim = 1024, weight_params_dim=256)
# mdl2 = CustomAttention(input_dim = 1024, weight_params_dim=256)

In [ ]:
wsi_coords = np.random.rand(50000, 2)  # Example coordinates, adjust as needed


In [ ]:
def compute_distances_in_batches(X, batch_size=1000, dtype=np.float32):
    n = X.shape[0]
    for i in range(0, n, batch_size):
        end = min(i + batch_size, n)
        batch_distances = pairwise_distances(X[i:end], X, metric='euclidean')
        yield batch_distances

# Usage
n_neighbors = 16
neighbor_indices1 = np.empty((wsi_coords.shape[0], n_neighbors), dtype=np.int32)

for idx, batch_patch_distance in enumerate(compute_distances_in_batches(wsi_coords)):
    start_idx = idx * 1000
    end_idx = min((idx + 1) * 1000, wsi_coords.shape[0])
    
    for i, distances in enumerate(batch_patch_distance):
        sorted_indices = np.argsort(distances)
        neighbor_indices1[start_idx + i] = sorted_indices[:n_neighbors]


In [ ]:
neighbor_indices1

In [ ]:
patch_distances = pairwise_distances(wsi_coords, metric='euclidean', n_jobs=1)
neighbor_indices2 = np.argsort(patch_distances, axis=1)[:, :16]

In [ ]:
neighbor_indices2

In [ ]:
assert neighbor_indices1 == neighbor_indices2

In [ ]:
# wsi_coords = h5py.File('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Patch/224_224/tcga_folder_4.TCGA-A8-A086-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2623.svs.h5', 'r')
# wsi_coords = wsi_coords['coords']
# wsi_coords2 = h5py.File('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Patch/224_224/tcga_folder_4.TCGA-AA-A01X-01Z-00-DX1.7433F54C-2A79-467A-8FEA-638AE48F42A0.svs.h5')

In [ ]:
# wsi_feats = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet18/tcga_folder_3.TCGA-A8-A086-01Z-00-DX1.2B52D1B8-5AD4-4BD6-ADF7-9D65B8EE2623.svs.pt')
# wsi_feats2 = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/resnet18/tcga_folder_4.TCGA-AA-A01X-01Z-00-DX1.7433F54C-2A79-467A-8FEA-638AE48F42A0.svs.pt')

In [ ]:
# features = []
# features.append(wsi_feats.cpu().numpy())
# features.append(wsi_feats2.cpu().numpy())

In [ ]:
# np.vstack(features).shape
# wsi_coords.shape
# wsi_feats.shape

In [ ]:
#--------------------------> init machine and person

machine_cohort_loc = '/Users/awxlong/Desktop/my-studies/hpc_exps/User/CRC_machine_config.pkl'#  f"{args.cohort_dir}/User/{args.localcohort_name}_machine_config.pkl"
with open(machine_cohort_loc, "rb") as f:   # Unpickling
    [data_locs,exp_locs,machine,user] = pickle.load(f)


    

In [ ]:
def save_hdf5(output_path, asset_dict, attr_dict=None, mode='a'):
    file = h5py.File(output_path, mode)
    for key, val in asset_dict.items():
        data_shape = val.shape
        if key not in file:
            data_type = val.dtype
            chunk_shape = (1,) + data_shape[1:]
            maxshape = (None,) + data_shape[1:]
            dset = file.create_dataset(key, shape=data_shape, maxshape=maxshape, chunks=chunk_shape, dtype=data_type)
            dset[:] = val
            if attr_dict is not None:
                if key in attr_dict.keys():
                    for attr_key, attr_val in attr_dict[key].items():
                        dset.attrs[attr_key] = attr_val
        else:
            dset = file[key]
            dset.resize(len(dset) + data_shape[0], axis=0)
            dset[-data_shape[0]:] = val
    file.close()
    return output_path

def compute_adj_coords(wsi_coords, wsi_feats, wsi_name, adj_coord_save_path, adj_matrix_save_path, force_recalc = False):
        # output_path_file = os.path.join(save_path + wsi_name + '.h5')
        # output_path_file = data_locs.abs_loc('feature') + f'{encoder}_adj_dictionary/{wsi_name}.h5'
        if not os.path.exists(f'{adj_matrix_save_path}{wsi_name}.pt') or force_recalc: 
             
            patch_distances = pairwise_distances(wsi_coords, metric='euclidean', n_jobs=1)
            neighbor_indices = np.argsort(patch_distances, axis=1)[:, :16]
            rows = np.asarray([[enum] * len(item) for enum, item in enumerate(neighbor_indices)]).ravel()
            columns = neighbor_indices.ravel()
            values = []
            coords = []
            for row, column in zip(rows, columns):
                    m1 = np.expand_dims(wsi_feats[int(row)], axis=0)
                    # pdb.set_trace()
                    m2 = np.expand_dims(wsi_feats[int(column)], axis=0)
                    value = distance.cdist(m1.reshape(1, -1), m2.reshape(1, -1), 'cosine')[0][0]
                    values.append(value)
                    coords.append((row, column))
            
            # mode = 'a'
            values = np.reshape(values, (wsi_coords.shape[0], neighbor_indices.shape[1]))
            
            coords = np.array(coords)
            
            asset_dict = {'adj_coords': coords, 'similarities': values, 'indices': neighbor_indices}

            save_hdf5(adj_coord_save_path, asset_dict, attr_dict=None)

            ### compute adjacency matrix
            values = np.nan_to_num(values)

            Idx = neighbor_indices[:, :8]
            rows = np.asarray([[enum] * len(item) for enum, item in enumerate(Idx)]).ravel()

            columns = Idx.ravel()

            neighbor_matrix = values[:, 1:]

            normalized_matrix = preprocessing.normalize(neighbor_matrix, norm="l2")

            similarities = np.exp(-normalized_matrix)

            values = np.concatenate((np.max(similarities, axis=1).reshape(-1, 1), similarities), axis=1)

            values = values[:, :8]

            values = values.ravel().tolist()

            sparse_coords= list(zip(rows, columns))

            # sparse_matrix = torch.sparse_coo_tensor(sparse_coords, values, (wsi_feats2.shape[0], wsi_feats2.shape[0]))

            indices = torch.tensor(sparse_coords, dtype=torch.long).t()
            # values = torch.tensor(values, dtype=torch.float32)
            values = torch.FloatTensor(values)
            sparse_matrix = torch.sparse.FloatTensor(indices, values, torch.Size([wsi_feats.shape[0], wsi_feats.shape[0]]))

            torch.save(sparse_matrix, f'{adj_matrix_save_path}{wsi_name}.pt')
            logger.info(f'Adjacency matrix stored at {adj_matrix_save_path}')
        else:
             logger.info(f'Adjacency matrix already exists at: {adj_matrix_save_path}{wsi_name}.pt')
        # return np.array(coords), values, neighbor_indices, sparse_matrix


In [ ]:
cohort_name = 'CRC'
idx = 1
local_cohort_dir = f'{exp_locs.root}Data/local_cohort_{cohort_name}_{idx}.csv'
df = pd.read_csv(local_cohort_dir)

step_size = 224
encoder = 'resnet18'

wsi_coord_root = data_locs.abs_loc('patch') + f'{step_size}_{step_size}/'
wsi_feats_root = data_locs.abs_loc('feature') + f'{encoder}/'

h5_path_root = data_locs.abs_loc('feature') + f'{encoder}_adj_dictionary/'
sparse_matrix_root = data_locs.abs_loc('feature') + f'{encoder}_adj_matrix/'

os.makedirs(h5_path_root, exist_ok=True)
os.makedirs(sparse_matrix_root, exist_ok=True)

for i in range(df.shape[0]):
    wsi_name = f'{df.loc[i, "folder"]}.{df.loc[i, "filename"]}'
    wsi_coords_name = f'{wsi_name}.h5'
    wsi_feats = f'{wsi_name}.pt'

    wsi_coords_dir = f'{wsi_coord_root}{wsi_coords_name}'
    wsi_feats_dir = f'{wsi_feats_root}{wsi_feats}'
    if os.path.exists(wsi_coords_dir) and os.path.exists(wsi_feats_dir):
            # Load the slide and its coordinates
            wsi_coordinates = h5py.File(wsi_coords_dir)
            wsi_coordinates = wsi_coordinates['coords']
            wsi_features = torch.load(wsi_feats_dir)  
            
            adj_coords_save_path = f'{h5_path_root}{wsi_name}.h5' # data_locs.abs_loc('feature') + f'{encoder}_adj_dictionary/{wsi_name}.h5'
            adj_matrix_save_path = f'{sparse_matrix_root}{wsi_name}.pt'
            
            # Process the slide and its coordinates
            compute_adj_coords(wsi_coords = wsi_coordinates, 
                               wsi_feats = wsi_features,
                               wsi_name = wsi_name,
                               adj_coord_save_path=adj_matrix_save_path,
                               adj_matrix_save_path=adj_coords_save_path,
                               force_recalc=False
                               )
            
             
    else:
            # Do nothing and continue to the next iteration
        continue

In [ ]:


adj_coords, similarities, neighbor_indices, sparse_matrix = compute_adj_coords(wsi_coords=wsi_coords2['coords'], wsi_feats=wsi_feats2)

        

In [ ]:
uni_feat = torch.rand((1, 29015, 1024))

In [ ]:
uni_adj_matrix = torch.load('/Users/awxlong/Desktop/my-studies/temp_data/CRC/Feature/uni_adj_matrix/temp_sparse_matrix.pt')
uni_adj_matrix

In [ ]:
from HistoMIL.MODEL.Image.MIL.CAMIL.model import CAMIL


In [ ]:
default_paras = CAMILParas()
default_paras.input_shape = 1024
rand_tensor = torch.rand(1, 1, 512)# .to('mps')
model = CAMIL(paras=default_paras)
model.to('cpu')

# uni_adj_matrix

In [ ]:
# mdl = Attention(dim=1024)

In [ ]:
out, alpha, k_alpha = model([uni_feat, uni_adj_matrix])

In [ ]:
out

In [ ]:
out
alpha
k_alpha.shape
out